В данной задаче, к сожалению, не сработали никакие из методов кластеризации, представленные в sklearn, а также не были найдены какие-либо способы кластеризации цепей Маркова, да и в целом не были найдены никакие зацепки, кроме алгоритма, представленного в [данной статье](https://habr.com/ru/companies/yandex/articles/461273/).

In [ ]:
import math
import random

with open('input.txt') as f:
    N, K = list(map(int, f.readline().split()))
    data = []
    alphabet = list(f.readline()) + ['']
    for _ in range(N):
        data.append(f.readline())

alphabet_len = len(alphabet)
data_len = len(data)
samples = [tuple([alphabet.index(token) for token in s] + [alphabet_len - 1, alphabet_len - 1]) for s in data]
probs = [random.random() for _ in range(data_len)]

def normalized_row(row):
    row_sum = sum(row) + 1e-9
    return [x / row_sum for x in row]

def normalized_matrix(matrix):
    return [normalized_row(row) for row in matrix]

for _ in range(150):
    old_probs = probs[:]
    p0, A = [0] * alphabet_len, [[0 for _ in range(alphabet_len)] for _ in range(alphabet_len)]
    q0, B = [0] * alphabet_len, [[0 for _ in range(alphabet_len)] for _ in range(alphabet_len)]
    for prob, sample in zip(probs, samples):
        p0[sample[0]] += prob
        q0[sample[0]] += 1 - prob
        for t1, t2 in zip(sample[:-1], sample[1:]):
            A[t1][t2] += prob
            B[t1][t2] += 1 - prob

    A, p0 = normalized_matrix(A), normalized_row(p0)
    B, q0 = normalized_matrix(B), normalized_row(q0)

    log_diff = [[math.log(b + 1e-9) - math.log(a + 1e-9) for b, a in zip(B_r, A_r)] for B_r, A_r in zip(B, A)]

    probs = [0] * data_len
    for i, sample in enumerate(samples):
        value = math.log(q0[sample[0]] + 1e-9) - math.log(p0[sample[0]] + 1e-9)
        for t1, t2 in zip(sample[:-1], sample[1:]):
            value += log_diff[t1][t2]
        probs[i] = 1.0 / (1.0 + math.exp(value))


for res in probs:
    print(int(res >= 0.5))